# bleu
> BLEU automated machine translation evaluation

In [ ]:
#| default_exp bleu

In [ ]:
#| hide
import os

In [ ]:
#| hide
running_in_colab = 'google.colab' in str(get_ipython())
homedir = ""
if running_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    homedir = "/content/drive/MyDrive"
else:
    homedir = os.getenv('HOME')

In [ ]:
#| hide
if running_in_colab:
    GITHUB_TEST_FOLDER = homedir+"/github/polyglottech/mteval"
    %cd {GITHUB_TEST_FOLDER}
    !pip3 install -e '.[dev]'

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from sacrebleu.metrics import BLEU
def measure_bleu(hypothesis_lines,reference_lines,targetlang,tok=None):
    """Measuring standard BLEU on set of hypothesis and references"""
    bleu = BLEU(trg_lang=targetlang,tokenize=tok)
    score = bleu.corpus_score(hypothesis_lines,[reference_lines])
    sig = bleu.get_signature().format(short=False)
    score_json = score.format(width=2,score_only=False,signature=sig,is_json=True)
    return score_json

In [ ]:
#| export
import csv
import json
import sys
from pathlib import Path
def measure_record_bleu(hypothesis_lines,reference_lines,sourcelang,targetlang,test_set_name,test_date,mtengine,score_pathname,score_fname,domain='',tok=None):
    """Score hypothesis with BLEU score and record it to a specified metrics file"""
    score_json = measure_bleu(hypothesis_lines,reference_lines,targetlang,tok)

    score_dict = json.loads(score_json)
    metric_record = score_dict
    metric_record["date"] = test_date
    metric_record["source_langid"] = sourcelang
    metric_record["target_langid"] = targetlang
    metric_record["test_set"] = test_set_name
    metric_record["engine"] = mtengine
    metric_record["domain"] = domain
    score_path = Path(score_pathname,score_fname)
    with open(score_path,"a",newline="") as score_file:
        field_names = metric_record.keys()
        writer = csv.DictWriter(score_file, fieldnames=field_names)
        if score_file.tell() == 0:
            writer.writeheader()
        writer.writerow(metric_record)

In [ ]:
#| hide
from mteval.dataset import *
import json
import os
from datetime import date

In [ ]:
#| hide
mteval_test_path = homedir+"/mteval_test/"
source_seg, ref_seg = download_read_set(mteval_test_path,"en","it","wmt09")
hyp_seg = ref_seg
score = json.loads(measure_bleu(hyp_seg,ref_seg,"it"))
# Only check a number of score results to verify a validly formatted result is returned
assert score['name'] == 'BLEU'
assert score['score'] == 100.0
assert score['nrefs'] == '1'

isodate = date.today().isoformat()
score_filename = "metrics_test.csv"
measure_record_bleu(hyp_seg,ref_seg,"en","it","wmt09",isodate,"dummy",mteval_test_path,score_filename)
assert os.path.exists(mteval_test_path+score_filename) == 1

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()